In [6]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 154.026512852
Epoch: 0002 cost= 56.308428974
Epoch: 0003 cost= 41.770156879
Epoch: 0004 cost= 33.933875737
Epoch: 0005 cost= 28.787246186
Epoch: 0006 cost= 25.071027294
Epoch: 0007 cost= 22.347714415
Epoch: 0008 cost= 20.101220662
Epoch: 0009 cost= 18.292646710
Epoch: 0010 cost= 16.852461527
Epoch: 0011 cost= 15.561708547
Epoch: 0012 cost= 14.507235367
Epoch: 0013 cost= 13.521578526
Epoch: 0014 cost= 12.629835881
Epoch: 0015 cost= 11.887300441
Optimization Finished!
Accuracy: 0.9123


Now, we have modified the code by adding in a gradient noise component. 

In [3]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import random_ops

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 50 # 1st layer number of features
n_hidden_2 =50 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# define gradient noise scale
gradient_noise_scale = 1.0

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

grads_and_vars = optimizer.compute_gradients(cost)
gradients, variables = zip(*grads_and_vars)
noisy_gradients = []
for gradient in gradients:
    if gradient is None:
      noisy_gradients.append(None)
      continue
    if isinstance(gradient, ops.IndexedSlices):
      gradient_shape = gradient.dense_shape
    else:
      gradient_shape = gradient.get_shape()
    noise = random_ops.truncated_normal(gradient_shape) * gradient_noise_scale
    noisy_gradients.append(gradient + noise)

noisy_grads_and_vars = list(zip(noisy_gradients, variables))
train_opt = optimizer.apply_gradients(noisy_grads_and_vars)


# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_opt, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 54.523476620
Epoch: 0002 cost= 19.107387671
Epoch: 0003 cost= 12.563198862
Epoch: 0004 cost= 9.273977405
Epoch: 0005 cost= 7.334051925
Epoch: 0006 cost= 5.946635454
Epoch: 0007 cost= 4.956802555
Epoch: 0008 cost= 4.211070791
Epoch: 0009 cost= 3.613363808
Epoch: 0010 cost= 3.108552761
Epoch: 0011 cost= 2.682546721
Epoch: 0012 cost= 2.340730362
Epoch: 0013 cost= 2.071511576
Epoch: 0014 cost= 1.852105742
Epoch: 0015 cost= 1.669174071
Optimization Finished!
Accuracy: 0.7114


We observe that when 100x gradient noise is added, the accuracy has gone down to 79%!

Below, we try to decrease the number of hidden units to 50, and the increase the number of layers by one. 

In [1]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import random_ops

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden = 50 # 1st layer number of features

n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# define gradient noise scale
gradient_noise_scale = 1.0

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    #Hidden layer, RELU Activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden])),
    'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden])),
    'h3': tf.Variable(tf.random_normal([n_hidden, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden])),
    'b2': tf.Variable(tf.random_normal([n_hidden])),
    'b3': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

grads_and_vars = optimizer.compute_gradients(cost)
gradients, variables = zip(*grads_and_vars)
noisy_gradients = []
for gradient in gradients:
    if gradient is None:
      noisy_gradients.append(None)
      continue
    if isinstance(gradient, ops.IndexedSlices):
      gradient_shape = gradient.dense_shape
    else:
      gradient_shape = gradient.get_shape()
    noise = random_ops.truncated_normal(gradient_shape) * gradient_noise_scale
    noisy_gradients.append(gradient + noise)

noisy_grads_and_vars = list(zip(noisy_gradients, variables))
train_opt = optimizer.apply_gradients(noisy_grads_and_vars)


# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_opt, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 85.304863670
Epoch: 0002 cost= 18.644972058
Epoch: 0003 cost= 9.587799523
Epoch: 0004 cost= 4.477712085
Epoch: 0005 cost= 2.272845787
Epoch: 0006 cost= 1.852295258
Epoch: 0007 cost= 1.714000374
Epoch: 0008 cost= 1.628495760
Epoch: 0009 cost= 1.589273282
Epoch: 0010 cost= 1.536517275
Epoch: 0011 cost= 1.510185187
Epoch: 0012 cost= 1.492731256
Epoch: 0013 cost= 1.481177086
Epoch: 0014 cost= 1.467025034
Epoch: 0015 cost= 1.438244873
Optimization Finished!
Accuracy: 0.5614


Now I will attempt 10 layer perceptron:

In [4]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''
import math
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import random_ops

# Parameters
learning_rate = 0.005
training_epochs = 15
batch_size = 200
display_step = 1

# Network Parameters
n_hidden = 50
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# define gradient noise scale
gradient_noise_scale = 1.0

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    #Hidden layer, RELU Activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    #Hidden layer, RELU Activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    #Hidden layer, RELU Activation
    layer_5 = tf.add(tf.matmul(layer_4, weights['h5']), biases['b5'])
    layer_5 = tf.nn.relu(layer_5)
    #Hidden layer, RELU Activation
    layer_6 = tf.add(tf.matmul(layer_5, weights['h6']), biases['b6'])
    layer_6 = tf.nn.relu(layer_6)
    #Hidden layer, RELU Activation
    layer_7 = tf.add(tf.matmul(layer_6, weights['h7']), biases['b7'])
    layer_7 = tf.nn.relu(layer_7)
    #Hidden layer, RELU Activation
    layer_8 = tf.add(tf.matmul(layer_7, weights['h8']), biases['b8'])
    layer_8 = tf.nn.relu(layer_8)
    #Hidden layer, RELU Activation
    layer_9 = tf.add(tf.matmul(layer_8, weights['h9']), biases['b9'])
    layer_9 = tf.nn.relu(layer_9)
    #Hidden layer, RELU Activation
    layer_10 = tf.add(tf.matmul(layer_9, weights['h10']), biases['b10'])
    layer_10 = tf.nn.relu(layer_10)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_10, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h3': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h4': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h5': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h6': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h7': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h8': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h9': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'h10': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], stddev = 1/math.sqrt(n_hidden))),
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b2': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b3': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b4': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b5': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b6': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b7': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b8': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b9': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'b10': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
    'out': tf.Variable(tf.random_normal([n_classes], stddev = 1/math.sqrt(n_hidden))),
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

grads_and_vars = optimizer.compute_gradients(cost)
gradients, variables = zip(*grads_and_vars)
noisy_gradients = []
for gradient in gradients:
    if gradient is None:
      noisy_gradients.append(None)
      continue
    if isinstance(gradient, ops.IndexedSlices):
      gradient_shape = gradient.dense_shape
    else:
      gradient_shape = gradient.get_shape()
    noise = random_ops.truncated_normal(gradient_shape) * gradient_noise_scale
    noisy_gradients.append(gradient + noise)

noisy_grads_and_vars = list(zip(noisy_gradients, variables))
train_opt = optimizer.apply_gradients(noisy_grads_and_vars)


# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_opt, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.310701294
Epoch: 0002 cost= 2.288274199
Epoch: 0003 cost= 2.130960203
Epoch: 0004 cost= 1.776757853
Epoch: 0005 cost= 1.447239249
Epoch: 0006 cost= 1.285714174
Epoch: 0007 cost= 1.196031783
Epoch: 0008 cost= 1.151221068
Epoch: 0009 cost= 1.152040341
Epoch: 0010 cost= 1.129889794
Epoch: 0011 cost= 1.123719214
Epoch: 0012 cost= 1.120449370
Epoch: 0013 cost= 1.119682016
Epoch: 0014 cost= 1.070964791
Epoch: 0015 cost= 1.081768620
Optimization Finished!
Accuracy: 0.6687


In [41]:
import math
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import random_ops


def percept10(noise, lr, n_hidden, numEpochs):

    mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

    # Parameters
    learning_rate = lr
    training_epochs = numEpochs
    batch_size = 200
    display_step = 1

    # Network Parameters
    n_hidden = 50
    n_input = 784 # MNIST data input (img shape: 28*28)
    n_classes = 10 # MNIST total classes (0-9 digits)

    # define gradient noise scale
    gradient_noise_scale = noise

    # tf Graph input
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])


    # Create model
    def multilayer_perceptron(x, weights, biases):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        #Hidden layer, RELU Activation
        layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
        layer_3 = tf.nn.relu(layer_3)
        #Hidden layer, RELU Activation
        layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
        layer_4 = tf.nn.relu(layer_4)
        #Hidden layer, RELU Activation
        layer_5 = tf.add(tf.matmul(layer_4, weights['h5']), biases['b5'])
        layer_5 = tf.nn.relu(layer_5)
        #Hidden layer, RELU Activation
        layer_6 = tf.add(tf.matmul(layer_5, weights['h6']), biases['b6'])
        layer_6 = tf.nn.relu(layer_6)
        #Hidden layer, RELU Activation
        layer_7 = tf.add(tf.matmul(layer_6, weights['h7']), biases['b7'])
        layer_7 = tf.nn.relu(layer_7)
        #Hidden layer, RELU Activation
        layer_8 = tf.add(tf.matmul(layer_7, weights['h8']), biases['b8'])
        layer_8 = tf.nn.relu(layer_8)
        #Hidden layer, RELU Activation
        layer_9 = tf.add(tf.matmul(layer_8, weights['h9']), biases['b9'])
        layer_9 = tf.nn.relu(layer_9)
        #Hidden layer, RELU Activation
        layer_10 = tf.add(tf.matmul(layer_9, weights['h10']), biases['b10'])
        layer_10 = tf.nn.relu(layer_10)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_10, weights['out']) + biases['out']
        return out_layer

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h3': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h4': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h5': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h6': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h7': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h8': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h9': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'h10': tf.Variable(tf.random_normal([n_hidden, n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes], stddev = 1/math.sqrt(n_hidden))),
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b2': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b3': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b4': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b5': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b6': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b7': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b8': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b9': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'b10': tf.Variable(tf.random_normal([n_hidden], stddev = 1/math.sqrt(n_hidden))),
        'out': tf.Variable(tf.random_normal([n_classes], stddev = 1/math.sqrt(n_hidden))),
    }

    # Construct model
    pred = multilayer_perceptron(x, weights, biases)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

    grads_and_vars = optimizer.compute_gradients(cost)
    gradients, variables = zip(*grads_and_vars)
    noisy_gradients = []
    for gradient in gradients:
        if gradient is None:
          noisy_gradients.append(None)
          continue
        if isinstance(gradient, ops.IndexedSlices):
          gradient_shape = gradient.dense_shape
        else:
          gradient_shape = gradient.get_shape()
        noise = random_ops.truncated_normal(gradient_shape) * gradient_noise_scale
        noisy_gradients.append(gradient + noise)

    noisy_grads_and_vars = list(zip(noisy_gradients, variables))
    train_opt = optimizer.apply_gradients(noisy_grads_and_vars)


    # Initializing the variables
    init = tf.initialize_all_variables()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)

        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size)
                # Run optimization op (backprop) and cost op (to get loss value)
                _, c = sess.run([train_opt, cost], feed_dict={x: batch_x,
                                                              y: batch_y})
                # Compute average loss
                avg_cost += c / total_batch
            if epoch % display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost=", \
                    "{:.9f}".format(avg_cost))
        print("Optimization Finished!")

        # Test model
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        acc = accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
        print("Accuracy:", acc)
        return acc

In [27]:
percept10(noise=1, lr =.005, n_hidden = 50)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Accuracy: 0.6914


0.69139999

In [12]:
percept10(noise=1, lr =.005, n_hidden = 50)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.305572053
Epoch: 0002 cost= 2.289341545
Epoch: 0003 cost= 2.067046631
Epoch: 0004 cost= 1.582955513
Epoch: 0005 cost= 1.257955252
Epoch: 0006 cost= 1.195730397
Epoch: 0007 cost= 1.191572426
Epoch: 0008 cost= 1.145896042
Epoch: 0009 cost= 1.146947614
Epoch: 0010 cost= 1.120890493
Epoch: 0011 cost= 1.123896382
Epoch: 0012 cost= 1.115336543
Epoch: 0013 cost= 1.109680106
Epoch: 0014 cost= 1.052895896
Epoch: 0015 cost= 1.099252642
Optimization Finished!
Accuracy: 0.6578


In [13]:
percept10(noise=0, lr =.005, n_hidden = 50)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.310609173
Epoch: 0002 cost= 2.300800451
Epoch: 0003 cost= 2.295851763
Epoch: 0004 cost= 2.291024674
Epoch: 0005 cost= 2.284386651
Epoch: 0006 cost= 2.273196532
Epoch: 0007 cost= 2.252023096
Epoch: 0008 cost= 2.209983734
Epoch: 0009 cost= 2.119852511
Epoch: 0010 cost= 1.942089891
Epoch: 0011 cost= 1.625608429
Epoch: 0012 cost= 1.250933658
Epoch: 0013 cost= 1.032171421
Epoch: 0014 cost= 0.853778077
Epoch: 0015 cost= 0.707395576
Optimization Finished!
Accuracy: 0.7643


In [22]:
def test1():
    noNoiseAcc = [0]*10
    noiseAcc = [0]*10
    
    #With no noise
    print("NO NOISE!!!")
    for i in range(10):
        noNoiseAcc[i] = percept10(noise=0, lr =.005, n_hidden = 50)
        
    #With noise
    print("NOISE!!!")
    for i in range(10):
        noiseAcc[i] = percept10(noise=1, lr =.005, n_hidden = 50)
        
    return noNoiseAcc, noiseAcc

In [23]:
noNoiseAcc, noiseAcc = test1()

NO NOISE!!!
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Accuracy: 0.8333
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Accuracy: 0.7024
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


...

Accuracy: 0.6465
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Accuracy: 0.6202
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Accuracy: 0.6642


In [24]:
noNoiseAcc

[<tf.Tensor 'Mean_21:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_23:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_25:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_27:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_29:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_31:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_33:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_35:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_37:0' shape=() dtype=float32>,
 <tf.Tensor 'Mean_39:0' shape=() dtype=float32>]

In [29]:
noNoise1 = [0.8333, 0.7024, 0.8671,  0.7777, 0.8362,  0.7132, 0.8217, 0.8577, 0.8409, 0.788]

In [30]:
noise1 = [0.7259, 0.6849, 0.6908, 0.7071, 0.6719, 0.6839,  0.692, 0.6465, 0.6202,  0.6642 ]

In [37]:
import numpy as np
print(np.mean(noNoise1))
print(np.mean(noise1))

0.80382
0.67874


We will now try the experiment at 30 epochs. 

In [42]:
def test2():
    noNoiseAcc = [0]*10
    noiseAcc = [0]*10
    
    #With no noise
    print("NO NOISE!!!")
    for i in range(10):
        noNoiseAcc[i] = percept10(noise=0, lr =.005, n_hidden = 50, numEpochs = 30)
        
    #With noise
    print("NOISE!!!")
    for i in range(10):
        noiseAcc[i] = percept10(noise=1, lr =.005, n_hidden = 50, numEpochs = 30)
        
    return noNoiseAcc, noiseAcc

In [43]:
noNoise2, noise2 = test2()

NO NOISE!!!
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.308922011
Epoch: 0002 cost= 2.297599703
Epoch: 0003 cost= 2.286727290
Epoch: 0004 cost= 2.270680265
Epoch: 0005 cost= 2.238699178
Epoch: 0006 cost= 2.156078348
Epoch: 0007 cost= 1.958667235
Epoch: 0008 cost= 1.638967583
Epoch: 0009 cost= 1.265686341
Epoch: 0010 cost= 1.029916122
Epoch: 0011 cost= 0.891710571
Epoch: 0012 cost= 0.794224846
Epoch: 0013 cost= 0.722688301
Epoch: 0014 cost= 0.665491140
Epoch: 0015 cost= 0.612503959
Epoch: 0016 cost= 0.573697970
Epoch: 0017 cost= 0.533992631
Epoch: 0018 cost= 0.502323254
Epoch: 0019 cost= 0.476191879
Epoch: 0020 cost= 0.445128516
Epoch: 0021 cost= 0.423382921
Epoch: 0022 cost= 0.403598210
Epoch: 0023 cost= 0.382726581
Epoch: 0024 cost= 0.367767693
Epoch: 0025 cost= 0.351580595
Epoch: 0026 cost= 0.336313111
Epoch: 0027 cost= 0.324204724
Epoch: 0028 cost= 0.315008204
Epoch: 0029 cost= 0.304267843
Epoch: 0030 cost= 0.291669736
Optimization Finished!
Accuracy: 0.9125
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.304845015
Epoch: 0002 cost= 2.296956547
Epoch: 0003 cost= 2.289803804
Epoch: 0004 cost= 2.280017806
Epoch: 0005 cost= 2.262494503
Epoch: 0006 cost= 2.221839924
Epoch: 0007 cost= 2.123042118
Epoch: 0008 cost= 1.895279951
Epoch: 0009 cost= 1.507793982
Epoch: 0010 cost= 1.170274974
Epoch: 0011 cost= 0.983230373
Epoch: 0012 cost= 0.859934184
Epoch: 0013 cost= 0.762356774
Epoch: 0014 cost= 0.687930419
Epoch: 0015 cost= 0.619427356
Epoch: 0016 cost= 0.554804626
Epoch: 0017 cost= 0.508926300
Epoch: 0018 cost= 0.471127832
Epoch: 0019 cost= 0.436788933
Epoch: 0020 cost= 0.414143350
Epoch: 0021 cost= 0.391532573
Epoch: 0022 cost= 0.374960563
Epoch: 0023 cost= 0.360029904
Epoch: 0024 cost= 0.349188700
Epoch: 0025 cost= 0.334947372
Epoch: 0026 cost= 0.324854024
Epoch: 0027 cost= 0.315317291
Epoch: 0028 cost= 0.306032270
Epoch: 0029 cost= 0.299012644
Epoch: 0030 cost= 0.290900849
Optimization Finished!
Accuracy: 0.9139
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /

...

Epoch: 0001 cost= 2.317090816
Epoch: 0002 cost= 2.290761114
Epoch: 0003 cost= 2.159735544
Epoch: 0004 cost= 1.815225685
Epoch: 0005 cost= 1.412881527
Epoch: 0006 cost= 1.261131846
Epoch: 0007 cost= 1.159865233
Epoch: 0008 cost= 1.110836744
Epoch: 0009 cost= 1.080139222
Epoch: 0010 cost= 1.052926565
Epoch: 0011 cost= 1.036031565
Epoch: 0012 cost= 1.065459845
Epoch: 0013 cost= 1.091242092
Epoch: 0014 cost= 1.050285395
Epoch: 0015 cost= 1.064126346
Epoch: 0016 cost= 1.063441148
Epoch: 0017 cost= 1.080360006
Epoch: 0018 cost= 1.079093887
Epoch: 0019 cost= 1.091964256
Epoch: 0020 cost= 1.060905920
Epoch: 0021 cost= 1.074315929
Epoch: 0022 cost= 1.085064983
Epoch: 0023 cost= 1.064146959
Epoch: 0024 cost= 1.168368346
Epoch: 0025 cost= 1.145672216
Epoch: 0026 cost= 1.132900299
Epoch: 0027 cost= 1.063124159
Epoch: 0028 cost= 1.116331348
Epoch: 0029 cost= 1.162306785
Epoch: 0030 cost= 1.206565112
Optimization Finished!
Accuracy: 0.2087
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.320119781
Epoch: 0002 cost= 2.281096938
Epoch: 0003 cost= 2.130687202
Epoch: 0004 cost= 1.770092670
Epoch: 0005 cost= 1.417077385
Epoch: 0006 cost= 1.250800603
Epoch: 0007 cost= 1.246729945
Epoch: 0008 cost= 1.201531047
Epoch: 0009 cost= 1.164333805
Epoch: 0010 cost= 1.165605288
Epoch: 0011 cost= 1.134926115
Epoch: 0012 cost= 1.107191471
Epoch: 0013 cost= 1.130351067
Epoch: 0014 cost= 1.124969048
Epoch: 0015 cost= 1.081597920
Epoch: 0016 cost= 1.099345089
Epoch: 0017 cost= 1.139881696
Epoch: 0018 cost= 1.084076910
Epoch: 0019 cost= 1.106666398
Epoch: 0020 cost= 1.084790694
Epoch: 0021 cost= 1.066434134
Epoch: 0022 cost= 1.111259714
Epoch: 0023 cost= 1.157989810
Epoch: 0024 cost= 1.126691394
Epoch: 0025 cost= 1.184591271
Epoch: 0026 cost= 1.183946586
Epoch: 0027 cost= 1.128889351
Epoch: 0028 cost= 1.126186270
Epoch: 0029 cost= 1.107085426
Epoch: 0030 cost= 1.096222491
Optimization Finished!
Accuracy: 0.6981
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch: 0001 cost= 2.313362448
Epoch: 0002 cost= 2.284222209
Epoch: 0003 cost= 1.986023432
Epoch: 0004 cost= 1.440327975
Epoch: 0005 cost= 1.238995430
Epoch: 0006 cost= 1.077197206
Epoch: 0007 cost= 1.021661461
Epoch: 0008 cost= 1.010177727
Epoch: 0009 cost= 1.040102865
Epoch: 0010 cost= 1.044605642
Epoch: 0011 cost= 1.040847797
Epoch: 0012 cost= 1.016792605
Epoch: 0013 cost= 1.041844055
Epoch: 0014 cost= 1.038224174
Epoch: 0015 cost= 1.048884718
Epoch: 0016 cost= 1.074321466
Epoch: 0017 cost= 1.079901201
Epoch: 0018 cost= 1.088014857
Epoch: 0019 cost= 1.107896463
Epoch: 0020 cost= 1.135693160
Epoch: 0021 cost= 1.136222536
Epoch: 0022 cost= 1.107042198
Epoch: 0023 cost= 1.105422416
Epoch: 0024 cost= 1.056073630
Epoch: 0025 cost= 1.097620462
Epoch: 0026 cost= 1.066625403
Epoch: 0027 cost= 1.125728829
Epoch: 0028 cost= 1.067173813
Epoch: 0029 cost= 1.122930715
Epoch: 0030 cost= 1.096482102
Optimization Finished!
Accuracy: 0.7043


In [44]:
noNoise2

[0.91250002,
 0.91390002,
 0.90850002,
 0.90460002,
 0.90850002,
 0.917,
 0.88450003,
 0.9102,
 0.91189998,
 0.90530002]

In [45]:
noise2

[0.69520003,
 0.65750003,
 0.097999997,
 0.097999997,
 0.65350002,
 0.6882,
 0.59990001,
 0.2087,
 0.69809997,
 0.70429999]